In [13]:
# Autoreload changed python modules
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import os, glob
import h5py # for loading .h5 files
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
plt.style.use('seaborn') # pretty matplotlib plots
plt.rcParams["axes.grid"] = False # don't show grid lines on plots by default
plt.rcParams['figure.figsize'] = (12, 16) # increase size of subplots

In [16]:
import cs230_project_utilities as utils
# make sure you have pywt: pip3 install PyWavelets --user

# Loading the data

In [17]:
# Location of directory H5Exports_AnimiX/ (downloaded from Olivier's link)
dataset_directory = '/scratch/groups/mwinterm/BoostMRI/data/H5Exports_AnimiX'

In [18]:
# Find all the files in our dataset
files = utils.automap.find_dataset_files(dataset_directory)

Found 133 .h5 files and 133 .txt files.


# Visualizing the data

In [20]:
# Finally, we can see the raw data
h5 = files['724_7094']['h5']
h5_data = utils.automap.read_h5_file(h5)
print(h5_data.keys())

images = h5_data['images']
magnitude = h5_data['magnitude']
phase = h5_data['phase']
classification = h5_data['classification']

# (Note: shape of magnitude and phase are different from image)
print(images.shape, magnitude.shape, phase.shape)
print(np.ndarray.flatten(classification))

(15, 256, 256)
dict_keys(['classification', 'images', 'magnitude', 'phase'])
(15, 256, 256) (15, 256, 240) (15, 256, 240)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


/home/users/awal/.local/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [28]:
# Uncomment to view example image in dataset
# utils.plot.imshowgray(utils.automap.transform_training_set_image_to_match_automap_output(images[0]))

In [24]:
# Construct FFT (k-space data) from magnitude and phase
fft = magnitude[0] * np.exp(1j * phase[0])

# Take the inverse FFT
ifft = utils.signal_processing.ifft_2D_centered(fft)

# Note: shape of magnitude and phase are different from image.
# Because of this, the reconstruction shape is different from the
# image shape and so we can't compare the image and reconstruction directly.
# How will we solve this?

# This check to make sure we are correctly combining magnitude and phase to construct the full, complex-valued FFT.
print('Error in FFT magnitude: {}'.format(utils.signal_processing.mean_square_error(np.abs(fft), magnitude[0])))
print('Error in FFT phase: {}'.format(utils.signal_processing.mean_square_error(np.angle(fft), phase[0])))

Error in FFT magnitude: 1.8354682870141233e-19
Error in FFT phase: 1.5035561514171795e-05


In [25]:
# Uncomment to show plots
# plt.subplot(2, 2, 1)
# plt.title('Reconstruction')
# utils.plot.imshowgray(np.abs(ifft))
# # 
# plt.subplot(2, 2, 2)
# plt.title('FFT (Magnitude)')
# utils.plot.imshowfft(np.abs(fft))

# plt.subplot(2, 2, 3)
# plt.title('Expected reconstruction')
# image = utils.automap.transform_training_set_image_to_match_automap_output(images[0])
# utils.plot.imshowgray(image)

# plt.subplot(2, 2, 4)
# plt.title('FFT (Phase)')
# utils.plot.imshowgray(np.angle(fft))

# Automap Model

In [10]:
import keras
from keras.layers import Input,Conv2D, Conv2DTranspose, Dense, Reshape, Flatten
from keras.models import Model
from keras.optimizers import RMSprop
from keras import losses

Using TensorFlow backend.


In [11]:
def load_model():
    res=64
    X = Input((res,res,2))
    X1 = Flatten()(X)
    fc1 = Dense(res * res * 2, activation = 'tanh')(X1)
    fc2 = Dense(res * res, activation = 'tanh')(fc1)
    fc3 = Dense(res * res, activation = 'tanh')(fc2)
    X2 = Reshape((res, res, 1))(fc3)
    conv1_1 = Conv2D(64, 5, activation='relu', padding='same')(X2)
    conv1_2 = Conv2D(64, 5, activation='relu', padding='same')(conv1_1)
    conv1_3 = Conv2DTranspose(64, 9, activation='relu', padding='same')(conv1_2)
    out = Conv2D(1, 1, activation = 'linear',padding='same')(conv1_3)
    model=Model(inputs=X, outputs=out)
    model.compile(optimizer=RMSprop(lr=1e-5), loss='mean_squared_error')
    return model

In [12]:
model = load_model()

Instructions for updating:
Colocations handled automatically by placer.
